<a href="https://colab.research.google.com/github/tegee/gpt-2/blob/master/KNGPT2_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

# Use language modeling version as of April 21st.
!git checkout b1ff0b2ae7d368b7db3a8a8472a29cc195d278d8

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')

!pip install dict2obj


Cloning into 'transformers'...
remote: Enumerating objects: 54705, done.
remote: Total 54705 (delta 0), reused 0 (delta 0), pack-reused 54705
Receiving objects: 100% (54705/54705), 40.80 MiB | 28.31 MiB/s, done.
Resolving deltas: 100% (38269/38269), done.
Note: checking out 'b1ff0b2ae7d368b7db3a8a8472a29cc195d278d8'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at b1ff0b2a Fix bug in examples: double wrap into DataParallel during eval
Processing /content/transformers
     |████████████████████████████████| 3.8MB 9.1MB/s 
     |████████████████████████████████| 133kB 45.2MB/s 
     |████████████████████████████████| 1.1MB 

In [ ]:
import torch
import run_language_modeling
import run_generation
from dict2obj import Dict2Obj
import collections
import random
import numpy as np

from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/KNGPT2oo' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/drive/MyDrive/KafiNnoonoo_corpus.txt \
    --do_eval \
    --eval_data_file=/content/drive/MyDrive/KANO-EVALUATION.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=60 \
    --gradient_accumulation_steps=5

In [ ]:
!ls '/content/drive/MyDrive/finetuned_models/KNGPT2oo'

checkpoint-1000  eval_results.txt	  tokenizer_config.json
checkpoint-500	 merges.txt		  training_args.bin
config.json	 special_tokens_map.json  vocab.json


In [ ]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = AutoModelWithLMHead.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None
  )
  
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

In [ ]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
#CHECKPOINT_PATH = "gpt2-medium"
CHECKPOINT_PATH = '/content/drive/MyDrive/finetuned_models/KNGPT2oo/checkpoint-1000'


# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/drive/MyDrive/all-kano-sos-eos.txt",
              "/content/drive/MyDrive/KANO-TEST.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = Dict2Obj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

12/07/2020 14:29:08 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/MyDrive/finetuned_models/KNGPT2oo/checkpoint-1000/config.json
12/07/2020 14:29:08 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

12/07/2020 14:29:08 

Running on device:  cuda


12/07/2020 14:29:49 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/MyDrive/finetuned_models/KNGPT2oo/checkpoint-1000/config.json
12/07/2020 14:29:49 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

12/07/2020 14:29:49 

8.051376342773438 is the perplexity of /content/drive/MyDrive/all-kano-sos-eos.txt according to /content/drive/MyDrive/finetuned_models/KNGPT2oo/checkpoint-1000


12/07/2020 14:33:54 - INFO - run_language_modeling -   ***** Running evaluation  *****
12/07/2020 14:33:54 - INFO - run_language_modeling -     Num examples = 433
12/07/2020 14:33:54 - INFO - run_language_modeling -     Batch size = 2
Evaluating: 100%|██████████| 217/217 [00:13<00:00, 15.73it/s]
12/07/2020 14:34:07 - INFO - run_language_modeling -   ***** Eval results  *****
12/07/2020 14:34:07 - INFO - run_language_modeling -     perplexity = tensor(97.2118)


97.21178436279297 is the perplexity of /content/drive/MyDrive/KANO-TEST.txt according to /content/drive/MyDrive/finetuned_models/KNGPT2oo/checkpoint-1000


In [ ]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [ ]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/KNGPT2oo/checkpoint-1000'

# You should try out other prompts as well as no prompt at all.
PROMPT = "kafi noonoo"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token='<|eos|>', # Set this if your dataset has a special word that indicates the end of a text.
  temperature=0.8,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=0.7,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=50,  # Number of tokens to generate.
  num_return_sequences=5,  # Number of independently computed samples to generate.
)
args = Dict2Obj(args)
model = load_model(args)
model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

12/07/2020 14:35:28 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/KNGPT2oo/checkpoint-1000/config.json
12/07/2020 14:35:28 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

12/07/2020 14:35:28

Running on device:  cuda


12/07/2020 14:36:12 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/KNGPT2oo/checkpoint-1000/config.json
12/07/2020 14:36:12 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

12/07/2020 14:36:12


====== GENERATION 0 ======
kafi noonooch gaacciyeete

====== GENERATION 1 ======
kafi noonooch kocciyeete; ebich doono yesuus kiristoosichi toommooch too'iyoonaa bi qelloonon wochii ciinnii beetina'one; biyoo bii

====== GENERATION 2 ======
kafi noonooch, ikka asho bulli giishacha ara asho bulli ikkooch giishacha ara asho bulli giishacha ara asho bulli ikkooch giis

====== GENERATION 3 ======
kafi noonooch wottaqqi shuunee wochoon bi toommooch too'ii beeti asheena'o aafoona yesheti wochoon bi qelli qelloona ittoshi qelloo

====== GENERATION 4 ======
kafi noonooch woyee qaaroona boono shuunoon gaacheebeet miheena'on kooriyeemmoch deggiye ebich hinnoona miheena'on shuu


In [ ]:
tag_start='<|sos|>'
while True:
    print('Input kafi-noonoo text or exit or "Enter" key for unconditional sampling.....')
    PROMPT = input(">>> ")
    if PROMPT == 'exit':
      break
    if PROMPT == '':
      PROMPT = tag_start 
    sequences = generate_samples(args, model, PROMPT)
    for idx, sequence in enumerate(sequences):
      print('\n====== GENERATION {} ======'.format(idx))
      print(sequence)
    #generate_samples(args, model, PROMPT)
      print('\n--------------------------------------------')
  #print('Thank you.')